In [ ]:
import cv2
import numpy as np
import face_recognition
import requests
import json


In [ ]:
cascPath = "haarcascade_frontalface_default.xml"
eyePath = "haarcascade_eye.xml"
smilePath = "haarcascade_smile.xml"

faceCascade = cv2.CascadeClassifier(cascPath)
eyeCascade = cv2.CascadeClassifier(eyePath)
smileCascade = cv2.CascadeClassifier(smilePath)

font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:

def parkFace():
    name = "Face"
    last_face = "Unknown"

    FaceData =[]
    Users = []

    r = requests.get(url = "http://192.168.43.34:5000/get_faces")
    if r.text !="None":
        r = json.loads(r.text)
        for i in r.keys():
            Users.append(i)
            l = []
            x = r[i][i][1:len(r[i])-2].split()
            for z in x:
                if z!="":
                    l.append(float(z))
            FaceData.append(l)


    if len(FaceData)>0:
        video_capture = cv2.VideoCapture(0)
        while True:

                process_this_frame =True
            # Capture frame-by-frame
                ret, frame = video_capture.read()

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                if process_this_frame:
                    face_locations = face_recognition.face_locations(frame)
                    face_encodings = face_recognition.face_encodings(frame, face_locations)
                # Initialize an array for the name of the detected users
                    face_names = []
                    for face_encoding in face_encodings:
                    # See if the face is a match for the known face(s)
                        matches = face_recognition.compare_faces(FaceData, face_encoding)
                        name = "Unknown"
                # check the known face with the smallest distance to the new face
                        face_distances = face_recognition.face_distance(FaceData, face_encoding)
                # Take the best one
                        best_match_index = np.argmin(face_distances)
                # If we have a match
                        if matches[best_match_index]:
                # Save the name of the best match
                            name = Users[best_match_index]
                        face_names.append(name)

                        if name !=last_face  and name !="Unknown":
                            last_face = name
                            video_capture.release()
                            cv2.destroyAllWindows()
                            return last_face
                            

                    process_this_frame = not process_this_frame   


                # Draw a rectangle around the faces
                    for i,(y, w, h, x) in enumerate(face_locations):

                        cv2.rectangle(frame, (x, y), (w, h), (255, 0, 0), 2)
                        roi_gray = gray[y:h, x:w]
                        roi_color = frame[y:h, x:w]
                        cv2.putText(frame,face_names[i],(x, y), font, 2,(255,0,0),3)

                        smile = smileCascade.detectMultiScale(
                            roi_gray,
                            scaleFactor= 1.16,
                            minNeighbors=35,
                            minSize=(25, 25),
                            flags=cv2.CASCADE_SCALE_IMAGE
                        )

                        for (sx, sy, sw, sh) in smile:
                            cv2.rectangle(roi_color, (sh, sy), (sx+sw, sy+sh), (255, 0, 0), 2)
                            cv2.putText(frame,'Smile',(x + sx,y + sy), 1, 1, (0, 255, 0), 1)

                        eyes = eyeCascade.detectMultiScale(roi_gray)
                        for (ex,ey,ew,eh) in eyes:
                            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                            cv2.putText(frame,'Eye',(x + ex,y + ey), 1, 1, (0, 255, 0), 1)

                cv2.putText(frame,'Number of Faces : ' + str(len(face_locations)),(40, 40), font, 1,(255,0,0),2)      
                # Display the resulting frame
                cv2.imshow('Video', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                  break


        # When everything is done, release the capture
        video_capture.release()
        cv2.destroyAllWindows()
    else:
        print("No records in database")
        

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2 # This is the OpenCV Python library
import re
import pytesseract # This is the TesseractOCR Python library

values = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 ")


def remover(my_string = ""):
      for item in my_string:
            if item not in values:
                  my_string = my_string.replace(item, "")
      return my_string

# Setup function to detect car plate
def carplate_detect(image):
    carplate_overlay = image.copy() 
    carplate_rects = carplate_haar_cascade.detectMultiScale(carplate_overlay,scaleFactor=1.1, minNeighbors=3)
    for x,y,w,h in carplate_rects: 
        cv2.rectangle(carplate_overlay, (x,y), (x+w,y+h), (255,0,0), 5) 
        
    return carplate_overlay

carplate_haar_cascade = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')
numbers={}

def carplate_detect(image):
    carplate_overlay = image.copy() 
    carplate_rects = carplate_haar_cascade.detectMultiScale(carplate_overlay,scaleFactor=1.1, minNeighbors=3)
    for x,y,w,h in carplate_rects: 
        cv2.rectangle(carplate_overlay, (x,y), (x+w,y+h), (255,0,0), 5) 
        
    return carplate_overlay

# Create function to retrieve only the car plate region itself
def carplate_extract(image):
    carplate_img=None
    carplate_rects = carplate_haar_cascade.detectMultiScale(image,scaleFactor=1.1, minNeighbors=5)
    for x,y,w,h in carplate_rects: 
            carplate_img = image[y+15:y+h-10 ,x+15:x+w-20] # Adjusted to extract specific region of interest i.e. car license plate

    return carplate_img

# Enlarge image for further processing later on
def enlarge_img(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return resized_image

def parknumber():
    lastplate=""
    video_capture = cv2.VideoCapture(0)
    count=0

    while True:
            number=""
            
            
            ret, frame = video_capture.read()

            carplate_rects = carplate_haar_cascade.detectMultiScale(frame,scaleFactor=1.1, minNeighbors=3)
            for x,y,w,h in carplate_rects: 
                cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 5) 
                
            for x,y,w,h in carplate_rects: 
                carplate_img = frame[y+15:y+h-10 ,x+15:x+w-20]
                number = pytesseract.image_to_string(carplate_img, 
                                  config = f'--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

            
                number = remover(number)
            if lastplate!=number:
                count+=1
                lastplate = number
                if number not in numbers:
                    numbers[number]=1
                else:
                    numbers[number]+=1
                
                if count==30:
                    mine=0
                    for i in numbers:
                        if len(i)>3 and numbers[i]>mine:
                            lastplate = i
                            mine=numbers[i]
                    video_capture.release()
                    cv2.destroyAllWindows()
                    return lastplate
                    
            cv2.imshow('Video', frame)
            
            

            if cv2.waitKey(1) & 0xFF == ord('q'):
              break


    # When everything is done, release the capture
    video_capture.release()
    cv2.destroyAllWindows()
    

In [ ]:
def parktype():
    import cv2
    cap = cv2.VideoCapture(0)  #Path to footage
    car_cascade = cv2.CascadeClassifier('cars.xml')  #Path to cars.xml
    bicycle_cascade= cv2.CascadeClassifier('motorbike.xml')  #Path to cars.xml

    while True:
        ret, img = cap.read()
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        cars=car_cascade.detectMultiScale(gray,1.8,2)
        bicycle = bicycle_cascade.detectMultiScale(gray, 1.8, 2)

        #Drawing rectangles on detected cars
        for (x,y,w,h) in cars:
            cv2.rectangle(img,(x,y),(x+w,y+h),(225,0,0),2)

        if len(cars)>0:
            cap.release()
            cv2.destroyAllWindows()
            return "1"

        # Drawing rectangles on detected bicycles
        for (x,y,w,h) in bicycle:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(img, 'Bike', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            plate = img[y:y + h, x:x + w]
            cv2.imshow('Bike', plate)

        cv2.imshow('img',img) #Shows the frame
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
class parkk:
    def __init__(self):
        pass
    def addface(self,face):
        self.face = face
    def addnum(self,num):
        self.num = num
    def addtype(self,t):
        self.type = t
class leavee:
    def __init__(self):
        pass
    def addface(self,face):
        self.face = face
    def addnum(self,num):
        self.num = num
    def addtype(self,t):
        self.type = t

In [ ]:
p = parkk()
l = leavee()

In [ ]:
from tkinter import *
import os
from datetime import datetime;


#creating instance of TK
root=Tk()

root.configure(background="black")


def park():
    if p.face and p.type and p.num:
        PARAMS = {'user':p.face,'regno':p.num,'type':p.type}
        r = requests.get(url = "http://192.168.43.34:5000/park",params = PARAMS)
        r = json.loads(r.text)
        if r == "-1":
            print("Sorry ! Parking slot not available")
        else:
            print("Allocated parking slot is "+ str(r))
        p.face=''
        p.type=''
        p.num =''
    else:
        print("Required 3 datas")
def leave():
    if l.face and l.type and l.num:
        PARAMS = {'user':l.face,'regno':l.num,'type':l.type}
        r = requests.get(url = "http://192.168.43.34:5000/leave",params = PARAMS)
        r = r.text
        print(r)
        l.face=''
        l.type=''
        l.num =''
    else:
        print("Required 3 datas")

def face():
    p.addface(parkFace())
    print("Parking : "+p.face+" (Mobile No)")
    
def num():
    p.addnum(parknumber())
    print("Parking : "+p.num+" (Vehicle No)")

def types():
    p.addtype(parktype())
    print("Parking : "+p.type+" (Vehicle Type)")
    
def face1():
    l.addface(parkFace())
    print("Leaving : "+l.face+" (Mobile No)")
    
def num1():
    l.addnum(parknumber())
    print("Leaving : "+l.num+" (Vehicle No)")

def types1():
    l.addtype(parktype())
    print("Leaving : "+l.type+" (Vehicle Type)")
  
 
def function6():

    root.destroy()

#stting title for the window
root.title("Smart Parking")

#creating a text label
Label(root, text="Face Detection, Vehicle Type Detection and Number Plate Recognition",font=("times new roman",20),fg="white",bg="black",height=2).grid(row=0,rowspan=2,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)

#creating first button
Button(root,text="Face Detecton (Park) ",font=("times new roman",20),bg="#000000",fg='green',command=face).grid(row=4,columnspan=2,sticky=W+E+N+S,padx=5,pady=5)

#creating second button
Button(root,text="Type Detection (Park) ",font=("times new roman",20),bg="#000000",fg='green',command=types).grid(row=5,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)

#creating third button
Button(root,text="Number Plate Recognition (Park)",font=('times new roman',20),bg="#000000",fg="green",command=num).grid(row=6,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)

Button(root,text="Park",font=('times new roman',20),bg="#000000",fg="green",command=park).grid(row=7,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)


#creating first button
Button(root,text="Face Detecton (Leave) ",font=("times new roman",20),bg="#000000",fg='red',command=face1).grid(row=8,columnspan=2,sticky=W+E+N+S,padx=5,pady=5)

#creating second button
Button(root,text="Type Detection (Leave) ",font=("times new roman",20),bg="#000000",fg='red',command=types1).grid(row=9,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)

#creating third button
Button(root,text="Number Plate Recognition (Leave)",font=('times new roman',20),bg="#000000",fg="red",command=num1).grid(row=10,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)

Button(root,text="Leave",font=('times new roman',20),bg="#000000",fg="red",command=leave).grid(row=11,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)


Button(root,text="Exit",font=('times new roman',20),bg="black",fg="gray",command=function6).grid(row=12,columnspan=2,sticky=N+E+W+S,padx=5,pady=5)

root.mainloop()
